In [4]:
!pip install -q langchain_google_genai langchain-community langchain-core requests duckduckgo-search

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
import requests

In [5]:
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [6]:
from google.colab import userdata
weather_api_key = userdata.get('WEATHER_API_KEY')

In [ ]:
!pip install -U ddgs

In [8]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [9]:
results = search_tool.invoke("Top Internation news Today")

In [10]:
results

"3 weeks ago - Listed below are other news channels aimed specifically at an international audience: ... US: Voice of America, CurrentTime TV(Russian), Alhurra (Arabic), TV Martí (Spanish) Bloomberg Television · India: NDTV 24x7, India Today , Mirror Now, NewsX, Republic World, Times Now Get the latest international news and world events from Asia, Europe, the Middle East, and more. See world news photos and videos at ABCNews.com Follow the latest international and world news , breaking stories and global current events from your trusted online news source. Movie review: 'Tron: Ares' falls back on derivative plot, themes Jane Goodall Institute vows to carry on its mission after her death U.S. News NPR world news , international art and culture, world business and financial markets, world economy, and global trends in health, science and technology. Subscribe to the World Story of the Day ..."

In [11]:
llm =ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=gemini_api_key
)

In [12]:
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--9db612a1-5327-4cc7-88b4-c5cf6b794559-0', usage_metadata={'input_tokens': 2, 'output_tokens': 221, 'total_tokens': 223, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 211}})

In [15]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key={weather_api_key}&query={city}'

  response = requests.get(url)

  return response.json()

In [16]:
get_weather_data("Dhaka")

/tmp/ipython-input-3699642282.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_weather_data("Dhaka")


{'request': {'type': 'City',
  'query': 'Dhaka, Bangladesh',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Dhaka',
  'country': 'Bangladesh',
  'region': '',
  'lat': '23.723',
  'lon': '90.409',
  'timezone_id': 'Asia/Dhaka',
  'localtime': '2025-10-09 20:03',
  'localtime_epoch': 1760040180,
  'utc_offset': '6.0'},
 'current': {'observation_time': '02:03 PM',
  'temperature': 29,
  'weather_code': 116,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0004_black_low_cloud.png'],
  'weather_descriptions': ['Partly Cloudy '],
  'astro': {'sunrise': '05:53 AM',
   'sunset': '05:38 PM',
   'moonrise': '07:09 PM',
   'moonset': '07:59 AM',
   'moon_phase': 'Waning Gibbous',
   'moon_illumination': 95},
  'air_quality': {'co': '306.85',
   'no2': '7.35',
   'o3': '158',
   'so2': '21.05',
   'pm2_5': '21.15',
   'pm10': '21.45',
   'us-epa-index': '2',
   'gb-defra-index': '2'},
  'wind_speed': 4,
  'wind_degree': 7,
  'wind_dir': 'N',
  

In [17]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [18]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt

In [19]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [20]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [23]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Bangladesh and Then tell me weather of this city"})
print(response)



> Entering new AgentExecutor chain...
Action: duckduckgo_search
Action Input: capital of BangladeshFrom Wikipedia, the free encyclopedia. Capital and largest city of Bangladesh . This article is about the capital city. Bangladesh , [a] officially the People's Republic of Bangladesh , [b] is a country in South Asia. It is the eighth-most populous country in the world and among the most densely populated with a population of over 171 million within an area of 148,460 square kilometres (57,320 sq mi). Exact time now, time zone, time difference, sunrise/sunset time and key facts for Dhaka, Bangladesh .Dhaka is the capital of Bangladesh . Latitude: 23.71. Longitude: 90.41. Bangladesh is a country in Asia, known for the Sundarbans mangroves and Bengal Delta. It has a population of 175.7 million, making it the 8th largest country in the world. Provides an overview of Bangladesh , including key dates and facts about this South Asian nation.People's republic of bangladesh : facts. Capital : D

In [24]:
response['output']

'The capital of Bangladesh is Dhaka. The current weather in Dhaka is Partly Cloudy with a temperature of 29°C, but it feels like 32°C. The wind speed is 4 km/h from the North, and the humidity is 70%.'